In [2]:
depolarize_probabilities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

results = {}

for depolarize_probability in depolarize_probabilities:
    results[depolarize_probability] = [0, 0]
    
# file = open("probability_results.txt", "w")
# file.write(str(results))
# file.close()

In [18]:
from ET_L23_qkd import QKD
from itertools import product

bits = 32
min_iterations = 1000
depolarize_probability = 0.5
rounds = 1

file = open("probability_results.txt", "r")
results = eval(file.read())
file.close()

actual_iterations = results[depolarize_probability][1]
if actual_iterations:
    actual_tolerability = results[depolarize_probability][0] / results[depolarize_probability][1]
else:
    actual_tolerability = 0

for i in range(actual_iterations, 100000):
    alice_key = ''
    while not alice_key:
        qkd = QKD(bits, rounds = rounds, depolarize_probability = depolarize_probability, efficient = True)
        alice_raw_key, bob_raw_key = qkd.generate_raw_key()
        alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)
    
    if len(alice_key) == 2:
        keys = list(set([ x + y for x, y in product(alice_key[0], alice_key[1]) ]))
    else:
        keys = list(set(alice_key[0]))
    
    for key in keys:
        if key == bob_key:
            results[depolarize_probability][0] += 1
            break
            
    results[depolarize_probability][1] += 1
    
    if (abs(actual_tolerability - (results[depolarize_probability][0] / results[depolarize_probability][1])) != 0) and (abs(actual_tolerability - (results[depolarize_probability][0] / results[depolarize_probability][1])) < 1e-4) and i > min_iterations:
        actual_tolerability = results[depolarize_probability][0] / results[depolarize_probability][1]
        print(f"Tolerability using {depolarize_probability * 100}% error and {results[depolarize_probability][1]} iterations: {actual_tolerability}")
        break

    actual_tolerability = results[depolarize_probability][0] / results[depolarize_probability][1]
    
    file = open("probability_results.txt", "w")
    file.write(str(results))
    file.close()

Tolerability using 50.0% error and 5002 iterations: 0.9130347860855658


#### Execute ET-L23

In [17]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from hashlib import sha256

from ET_L23_qkd import QKD

bits = 32
rounds = 1
qkd = QKD(bits, rounds = rounds, depolarize_probability = 0.5, efficient = True)

alice_raw_key, bob_raw_key = qkd.generate_raw_key()
print(f"Alice raw key: {alice_raw_key}")
print(f"Bob raw key: {bob_raw_key}")

alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)

if rounds == 2:
    keys = list(set([ x + y for x, y in zip(alice_key[0], alice_key[1]) ]))
else:
    keys = list(set(alice_key[0]))

for key in keys:
    if key == bob_key:
        print("QKD completed!")
        print(f"Shared key: {key}")

        alice_key = sha256(key.encode()).digest()
        bob_key = sha256(bob_key.encode()).digest()

        MSG = b"H3xTEL{R3v3rS3_R3C0nc1l14710N_1s_P34K!}"
        cipher = AES.new(alice_key, AES.MODE_ECB)
        encrypted = cipher.encrypt(pad(MSG, 16))

        print(f"Alice encrypted message: {encrypted.hex()}")

        cipher = AES.new(bob_key, AES.MODE_ECB)
        decrypted = unpad(cipher.decrypt(encrypted), 16)

        print(f"Bob decrypted message: {decrypted.decode()}")

Alice raw key: 10011110110101110000011101101011
Bob raw key: 10001110100100000010011010110001
